In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import IsolationForest
import xgboost as xgb
import numpy as np

# 데이터 읽기
museum_file_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/1. 문화시설/박물관전처리.csv'
culture_place_file_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/1. 문화시설/실내문화공간전처리.csv'
forest_file_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/2. 자연환경/forest_count_data.csv'
beach_file_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/2. 자연환경/modified_beach_data.csv'
accommodation_file_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/3. 숙박 시설/Final_Region_Facility_Counts_with_Renamed_Columns.csv'
shop_file_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/4. 편의 시설/아이 동반 쇼핑시설 지역별 시설 개수.csv'
park_file_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/4. 편의 시설/주차장 지역별 개수.csv'
play_fille_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/4. 편의 시설/아이 동반 놀이 시설 지역별 개수.csv'
trip_file_path = '/content/drive/MyDrive/문화 공모전/전처리 완료 데이터/국민여행조사 지역별 방문횟수 데이터.csv'

museum_data = pd.read_csv(museum_file_path)
culture_place_data = pd.read_csv(culture_place_file_path)
forest_data = pd.read_csv(forest_file_path)
beach_data = pd.read_csv(beach_file_path)
accommodation_data = pd.read_csv(accommodation_file_path)
shop_data = pd.read_csv(shop_file_path)
park_data = pd.read_csv(park_file_path)
play_data = pd.read_csv(play_fille_path)
data = pd.read_csv(trip_file_path)

# 데이터프레임 리스트 생성
dataframes = [museum_data, culture_place_data, forest_data, beach_data,
              accommodation_data, shop_data, park_data, play_data, data]

# 병합할 때 사용하는 함수
def merge_dataframes_on_region(df_list):
    # 가장 많은 지역명을 가진 데이터프레임을 기준으로 병합 시작
    merged_df = df_list[0]
    for df in df_list[1:]:
        merged_df = pd.merge(merged_df, df, on='지역', how='outer', suffixes=('', '_drop'))
        print(merged_df.head())
    return merged_df.fillna(0)  # 결측값을 0으로 채우기

# 데이터 병합 (지역을 기준으로)
facilities_data = merge_dataframes_on_region(dataframes)

# 중간 결과 확인
print(facilities_data.head())
print(f"Number of columns after merging: {len(facilities_data.columns)}")


# 불필요한 열 제거
if 'Unnamed: 0' in facilities_data.columns:
    facilities_data.drop(columns=['Unnamed: 0'], inplace=True)

# 컬럼명 변경
facilities_data.columns = ['지역', '박물관 개수', '실내 문화공간 개수',  '휴양림 개수', '해변 개수', '숙소 개수',
                           '아이 동반 쇼핑 시설 개수', '주차장 개수', '아이 동반 놀이 시설 개수', '여행건 개수']

# 독립변수(X) 설정
X = facilities_data[['박물관 개수', '실내 문화공간 개수',  '휴양림 개수', '해변 개수', '숙소 개수',
                     '아이 동반 쇼핑 시설 개수', '주차장 개수', '아이 동반 놀이 시설 개수', ]]


X

       지역    개수  개수_drop
0  강원 강릉시  41.0     39.0
1  강원 고성군  11.0      6.0
2  강원 동해시   3.0     11.0
3  강원 삼척시   5.0     15.0
4  강원 속초시   4.0     17.0
       지역    개수  개수_drop  개수_drop
0  강원 강릉시  41.0     39.0      1.0
1  강원 고성군  11.0      6.0      NaN
2  강원 동해시   3.0     11.0      NaN
3  강원 삼척시   5.0     15.0      1.0
4  강원 속초시   4.0     17.0      NaN
       지역    개수  개수_drop  개수_drop  개수_drop
0  강원 강릉시  41.0     39.0      1.0     20.0
1  강원 고성군  11.0      6.0      NaN     17.0
2  강원 동해시   3.0     11.0      NaN      5.0
3  강원 삼척시   5.0     15.0      1.0     28.0
4  강원 속초시   4.0     17.0      NaN      2.0
       지역    개수  개수_drop  개수_drop  개수_drop  개수_drop
0  강원 강릉시  41.0     39.0      1.0     20.0    734.0
1  강원 고성군  11.0      6.0      NaN     17.0    235.0
2  강원 동해시   3.0     11.0      NaN      5.0    114.0
3  강원 삼척시   5.0     15.0      1.0     28.0    181.0
4  강원 속초시   4.0     17.0      NaN      2.0    296.0
       지역    개수  개수_drop  개수_drop  개수_drop  개수_drop  개수_drop
0  강원 강릉시  41.0

,박물관 개수,실내 문화공간 개수,휴양림 개수,해변 개수,숙소 개수,아이 동반 쇼핑 시설 개수,주차장 개수,아이 동반 놀이 시설 개수
0,41.0,39.0,1.0,20.0,734.0,98.0,120.0,42.0
1,11.0,6.0,0.0,17.0,235.0,3.0,23.0,8.0
2,3.0,11.0,0.0,5.0,114.0,38.0,24.0,31.0
3,5.0,15.0,1.0,28.0,181.0,18.0,12.0,21.0
4,4.0,17.0,0.0,2.0,296.0,44.0,24.0,25.0
...,...,...,...,...,...,...,...,...
305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# 2

In [3]:
# 종속변수(y) 설정
y = facilities_data['여행건 개수']

# 데이터 전처리 (결측치 처리, 스케일링 등)
X.fillna(X.mean(), inplace=True)

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# XGBoost 회귀 모델 생성 및 학습
xgbr = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
xgbr.fit(X_train, y_train)

# 모델 예측
y_pred = xgbr.predict(X_test)

# 성능 평가
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 결과 출력
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

MAE: 38.871711049424185
MSE: 6545.198706688493
RMSE: 80.90240235424714
R2 Score: 0.7476285283163818


<ipython-input-3-ccaa6c37b727>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


In [4]:
# 각 독립변수의 중요도 추출
feature_importances = xgbr.feature_importances_

# 중요도를 데이터프레임으로 변환
importance_df = pd.DataFrame({'Variable': X.columns, 'Importance': feature_importances})

# 중요도 정렬
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(importance_df)

         Variable  Importance
4           숙소 개수    0.680193
6          주차장 개수    0.091659
3           해변 개수    0.063785
1      실내 문화공간 개수    0.055858
7  아이 동반 놀이 시설 개수    0.034930
0          박물관 개수    0.031203
2          휴양림 개수    0.025174
5  아이 동반 쇼핑 시설 개수    0.017196


# 1

In [3]:
# 종속변수(y) 설정
y = facilities_data['여행건 개수']

# 데이터 전처리 (결측치 처리, 스케일링 등)
X.fillna(X.mean(), inplace=True)

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 설정
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

xgbr = xgb.XGBRegressor(objective='reg:squarederror')

# 그리드 서치
grid_search = GridSearchCV(estimator=xgbr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
xgbr = xgb.XGBRegressor(**best_params)
xgbr.fit(X_train, y_train)


# 모델 예측
y_pred = xgbr.predict(X_test)

# 성능 평가
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 결과 출력
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

Fitting 3 folds for each of 2187 candidates, totalling 6561 fits


<ipython-input-3-3ea5085a8e2a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


Best parameters found:  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}
MAE: 37.0150769455058
MSE: 6025.95216765097
RMSE: 77.62700669001073
R2 Score: 0.7676497712298495


In [4]:
# 각 독립변수의 중요도 추출
feature_importances = xgbr.feature_importances_

# 중요도를 데이터프레임으로 변환
importance_df = pd.DataFrame({'Variable': X.columns, 'Importance': feature_importances})

# 중요도 정렬
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(importance_df)

         Variable  Importance
4           숙소 개수    0.371332
3           해변 개수    0.161624
1      실내 문화공간 개수    0.103563
5  아이 동반 쇼핑 시설 개수    0.087204
0          박물관 개수    0.081365
2          휴양림 개수    0.071251
6          주차장 개수    0.062697
7  아이 동반 놀이 시설 개수    0.060964


# 3

In [9]:
# 종속변수(y) 설정
y = facilities_data['여행건 개수']

# 이상치 제거
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X)
mask = yhat != -1
X, y = X[mask], y[mask]

# 데이터 전처리 (결측치 처리, 스케일링 등)
X.fillna(X.mean(), inplace=True)

# 데이터 스케일링
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 설정
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

xgbr = xgb.XGBRegressor(objective='reg:squarederror')

# 그리드 서치
grid_search = GridSearchCV(estimator=xgbr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
xgbr = xgb.XGBRegressor(**best_params)
xgbr.fit(X_train, y_train)

# 모델 예측
y_pred = xgbr.predict(X_test)

# 성능 평가
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 결과 출력
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

Fitting 3 folds for each of 2187 candidates, totalling 6561 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
<ipython-input-9-84c409d20e64>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


Best parameters found:  {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 1.0}
MAE: 21.41981953009963
MSE: 1168.3649619000876
RMSE: 34.181354009168324
R2 Score: 0.5424154493399947


In [10]:
# 각 독립변수의 중요도 추출
feature_importances = xgbr.feature_importances_

# 중요도를 데이터프레임으로 변환
importance_df = pd.DataFrame({'Variable': X.columns, 'Importance': feature_importances})

# 중요도 정렬
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(importance_df)

         Variable  Importance
4           숙소 개수    0.576293
3           해변 개수    0.134409
6          주차장 개수    0.069284
2          휴양림 개수    0.057308
7  아이 동반 놀이 시설 개수    0.049639
1      실내 문화공간 개수    0.045882
0          박물관 개수    0.044941
5  아이 동반 쇼핑 시설 개수    0.022243


# 4

In [14]:
# 종속변수(y) 설정
y = facilities_data['여행건 개수']

# 이상치 제거
best_contamination = 0
best_score = -np.inf
for contamination in [0.01, 0.05, 0.1, 0.2]:
    iso = IsolationForest(contamination=contamination, random_state=42)
    yhat = iso.fit_predict(X)
    mask = yhat != -1
    X_filtered, y_filtered = X[mask], y[mask]

    # 데이터 전처리 (결측치 처리, 스케일링 등)
    X_filtered.fillna(X_filtered.mean(), inplace=True)

    # 데이터 스케일링
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X_filtered)

    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_filtered, test_size=0.2, random_state=42)

    # 모델 학습
    xgbr = xgb.XGBRegressor(objective='reg:squarederror')
    xgbr.fit(X_train, y_train)

    # 모델 예측
    y_pred = xgbr.predict(X_test)

    # 성능 평가
    r2 = r2_score(y_test, y_pred)
    if r2 > best_score:
        best_score = r2
        best_contamination = contamination

print(f"Best contamination: {best_contamination}")
print(f"Best R2 Score: {best_score}")

# 최적의 contamination으로 이상치 제거
iso = IsolationForest(contamination=best_contamination, random_state=42)
yhat = iso.fit_predict(X)
mask = yhat != -1
X, y = X[mask], y[mask]

# 데이터 전처리 (결측치 처리, 스케일링 등)
X.fillna(X.mean(), inplace=True)

# 데이터 스케일링
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 설정
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

xgbr = xgb.XGBRegressor(objective='reg:squarederror')

# 그리드 서치
grid_search = GridSearchCV(estimator=xgbr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
xgbr = xgb.XGBRegressor(**best_params)
xgbr.fit(X_train, y_train)

# 모델 예측
y_pred = xgbr.predict(X_test)

# 성능 평가
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 결과 출력
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
<ipython-input-14-f02e11dfeef3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_filtered.fillna(X_filtered.mean(), inplace=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
<ipython-input-14-f02e11dfeef3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_filtered.fillna(X_filtered.mean(), inplace=True)
/usr

Best contamination: 0.1
Best R2 Score: 0.5599852963391945


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
<ipython-input-14-f02e11dfeef3>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


Fitting 3 folds for each of 2187 candidates, totalling 6561 fits
Best parameters found:  {'colsample_bytree': 0.9, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.8}
MAE: 15.52554985801024
MSE: 597.8109127295427
RMSE: 24.450172038853687
R2 Score: 0.5365036950648725


In [15]:
# 각 독립변수의 중요도 추출
feature_importances = xgbr.feature_importances_

# 중요도를 데이터프레임으로 변환
importance_df = pd.DataFrame({'Variable': X.columns, 'Importance': feature_importances})

# 중요도 정렬
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(importance_df)

         Variable  Importance
4           숙소 개수    0.502469
2          휴양림 개수    0.089356
0          박물관 개수    0.080360
1      실내 문화공간 개수    0.074801
7  아이 동반 놀이 시설 개수    0.074287
3           해변 개수    0.073013
5  아이 동반 쇼핑 시설 개수    0.066744
6          주차장 개수    0.038970


# 5

In [18]:
# 종속변수(y) 설정
y = facilities_data['여행건 개수']

# 이상치 제거
best_contamination = 0
best_score = -np.inf
for contamination in [0.01, 0.05, 0.1, 0.2]:
    iso = IsolationForest(contamination=contamination, random_state=42)
    yhat = iso.fit_predict(X)
    mask = yhat != -1
    X_filtered, y_filtered = X[mask], y[mask]

    # 데이터 전처리 (결측치 처리, 스케일링 등)
    X_filtered.fillna(X_filtered.mean(), inplace=True)

    # 데이터 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_filtered)

    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_filtered, test_size=0.2, random_state=42)

    # 모델 학습
    xgbr = xgb.XGBRegressor(objective='reg:squarederror')
    xgbr.fit(X_train, y_train)

    # 모델 예측
    y_pred = xgbr.predict(X_test)

    # 성능 평가
    r2 = r2_score(y_test, y_pred)
    if r2 > best_score:
        best_score = r2
        best_contamination = contamination

print(f"Best contamination: {best_contamination}")
print(f"Best R2 Score: {best_score}")

# 최적의 contamination으로 이상치 제거
iso = IsolationForest(contamination=best_contamination, random_state=42)
yhat = iso.fit_predict(X)
mask = yhat != -1
X, y = X[mask], y[mask]

# 데이터 전처리 (결측치 처리, 스케일링 등)
X.fillna(X.mean(), inplace=True)

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 설정
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

xgbr = xgb.XGBRegressor(objective='reg:squarederror')

# 그리드 서치
grid_search = GridSearchCV(estimator=xgbr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
xgbr = xgb.XGBRegressor(**best_params)
xgbr.fit(X_train, y_train)

# 모델 예측
y_pred = xgbr.predict(X_test)

# 성능 평가
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 결과 출력
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
<ipython-input-18-70976cf8846f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_filtered.fillna(X_filtered.mean(), inplace=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
<ipython-input-18-70976cf8846f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_filtered.fillna(X_filtered.mean(), inplace=True)
/usr

Best contamination: 0.1
Best R2 Score: 0.5599852963391945
Fitting 3 folds for each of 2187 candidates, totalling 6561 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
<ipython-input-18-70976cf8846f>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


Best parameters found:  {'colsample_bytree': 0.9, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.8}
MAE: 15.52554985801024
MSE: 597.8109127295427
RMSE: 24.450172038853687
R2 Score: 0.5365036950648725


In [19]:
# 각 독립변수의 중요도 추출
feature_importances = xgbr.feature_importances_

# 중요도를 데이터프레임으로 변환
importance_df = pd.DataFrame({'Variable': X.columns, 'Importance': feature_importances})

# 중요도 정렬
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(importance_df)

         Variable  Importance
4           숙소 개수    0.502469
2          휴양림 개수    0.089356
0          박물관 개수    0.080360
1      실내 문화공간 개수    0.074801
7  아이 동반 놀이 시설 개수    0.074287
3           해변 개수    0.073013
5  아이 동반 쇼핑 시설 개수    0.066744
6          주차장 개수    0.038970
